## Pandas 使用笔记

Python 中处理二维表数据的核心， Python 中的 Excel


In [ ]:
import os
import re

import numpy as np # 当然少不了和 numpy 搭配使用
import pandas as pd

### 纵向合并某一文件夹内所有的 Excel 文件

经典任务，实现方法有很多，但就是还没有一个统一简洁的官方方法

In [ ]:
def MERGE(workdir):
    # 获取路径内全部文件，这里假设文件夹内有且只有目标的excel文件。
    # 参杂其他东西也是有办法的，但这里先不涉及那么多
    filenames = os.listdir(workdir)
    filepath = []
    for i in range(0,len(filenames)):
        filepath.append(os.path.join(workdir,filenames[i]))
        
    # 遍历获取所有 excel 内容
    content = []
    
    for file in filepath:
        # pandas 读取 excel
        # 根据需要可以增加参数:
        #   sheet_name 指定工作簿 
        #   skipfooter 忽略尾部 
        #   dtype 指定列变量类型，详见文档
        fileread = pd.read_excel(file)

        # 全部加入 content 列表中
        content.append(fileread)
    
    # 将 content 列表中的内容合并为 DataFrame
    result = pd.concat(content) # concat 默认为纵向合并
    
    return result

### 横向合并

同样有多种方法，可以实现类似 sql 的各种 join

In [ ]:
# Merge方法，还有更多的合并参数可以选择，详见文档
df3 = df1.merge(df2, how='outer', on='column_name') # 指定合并根据列
df3 = df1.merge(df2, how='outer', left_index=True, right_index=True) # 根据两侧索引合并

# Join方法，常用于分割列后的返回合并
split = df['column_name'].str.split(expand=True) # 默认按空格分割，且展开列
df = df.join(split)

### 更多具体操作

In [ ]:
df = MERGE('filepath')

# 重设 pandas 默认索引，常用于合并多个 dataframe 之后
df.reset_index(drop=True)

# 使用某一列作为索引
df.set_index([''], inplace=True)

# 输出，可选参数众多，有需要时详见 IO 文档
df.to_excel('filename', sheet_name='', index=True, index_label='', merge_cells=False) # 分割合并单元格
df.to_csv('filename')

# 去除行/列
df.drop(columns=['column_name'], inplace=True) # 关键在于 inplace，此参数默认为 false，只会生成一个新的对象
df.drop(['column_name'], axis=0, inplace=True) # 效果同上，axis=0 即列

# 赋值新建列
df['new_column_name'] = 'new_value'
df['index1'] = list(range(1, len(allmerge['index'])+1)) # 给一个list，新建一个从1开始的索引

# 按某列取值筛选行（类似 excel 的筛选），可组合条件使用
df[df[('column_name1']=='value1'&df['column_name2']=='value2')|df['column_name3'=='value3']
df = df[df['column_name']!='value'&-df['column_name'].str.contains('str1')&-df['column_name'].str.match('str2')]
# 注意上面的负号，可以对布尔值取补集

# 给定规则替换值，至少有 map 和 replace 两种方法
namemap = {
'被替换值':'替换值',
'中华人民共和国':'中国',
}
df['column_name'] = df['column_name'].map(namemap) # map 方法没有 inplace 参数，只能重新赋值
# map 方法还可以使用函数，更名副其实。详见文档

# replace 方法，这里可以配合正则使用
pattern1 = r'.*身份证$'
pattern2 = r'.*签证$'
# regex 参数用于启动正则模式
df['column_name'].replace( [pattern1, pattern2], ['身份证', '签证'], regex=True, inplace=True)

# 处理文本两侧可能有的多余空格
df['column_name'] = df['column_name'].str.lstrip() # 左去除
df['column_name'] = df['column_name'].str.rstrip() # 右去除

# 处理 csv 中常见的引号文本标记
df['column_name']=df['column_name'].str.strip('\'')

# 列改名
df.rename(columns = {'old_name':'new_name'}, inplace=True)

# 缺失值 nan 需要用 numpy 处理
df.replace(np.nan, 0, inplace = True)

# 索引筛选，假设为时间序列数据。
# 这一部分方法众多，特别是涉及到多层行/列结构的时候，需要时精读文档
df = df.loc['2018-02-08':]

### 日期处理

日期列读入 pandas 后往往只是一般的 object 类型，需要先转换成 datetime 类型方便后续使用

In [ ]:
# Datetime 日期格式转换。
df['new_datetime'] = pd.to_datetime(df['old_datetime'])
# 一次转换多列
df[['time1', 'time2']] = df[['time1', 'time2']].apply(pd.to_datetime)

# Datetime 日期拆解
df['date'] = df['new_datetime'].dt.date
df['time'] = df['new_datetime'].dt.time
df['secs'] = df['datetime1'] - df['datetime2']
df['secs'] = df['secs'].dt.total_seconds()
df[['secs1','secs2']] = df[['secs1','secs2']].astype('timedelta64[h]') # 根据需要调整 timedelta 类型

### 数据透视表

类似 Excel 的效果

In [ ]:
# pivot 数据透视表

df_pivot = df.pivot_table(values=['value_column_name'], 
                          index=['row_column_name'], 
                          columns=['column_column_name', 
                                   aggfunc='pd.Series.nunique', 
                                   margins=True)
# 参数：
#   Pandas  | Excel
#   values  |  值
#   index   |  行
#   columns |  列
#   aggfunc | 汇总方法
#  margins-是否显示加总行/列

# pivot 表 stack
df_pivot = df_pivot.stack().to_frame() # 小心 stack()方法生成的是个 series 对象，还需要进一步转化为 dataframe